# XArrayDataset

In [1]:
import autoroot
from oceanbench._src.datasets.base import XArrayDataset, XRConcatDataset
import numpy as np
import pandas as pd
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)
from xarray_dataclasses import asdataarray, asdataset
%load_ext autoreload
%autoreload 2


/Users/eman/miniconda3/envs/oceanbench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, CoordinateAxis,
    TimeAxis, SSH2D, SSH2DT, Bounds
)
from xarray_dataclasses import asdataarray, asdataset, Data, Name, Coordof, Coord
from oceanbench._src.geoprocessing.gridding import create_coord_grid

In [3]:
from dataclasses import dataclass
from typing import Literal, Tuple


X = Literal["x"]
Y = Literal["y"]
Z = Literal["z"]

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable2D:
    data: Data[tuple[X,Y], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable3D:
    data: Data[tuple[X, Y, Z], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    z: Coord[Z, np.ndarray] = 0
    name: Name[str] = "var"



**Cases I**: 1D Time Series

* `[360] * [30p] * [30s] = [12]`
* Latent Vector - `[12, Z]`
* Ensemble Predictions - `[12, 20]`

**Case II**: Lat/Lon

* `[200, 200] * [10, 10] = [20, 20]`

**Case III**: Lat/Lon/Time

* `[200, 200, 360] * [10, 10, 30] = [20, 20, 12]`
* 360 Days to 30 Days
* Latent Vector
* Ensemble Predictions

## 1D Temporal Field

We have a 1D time series:



In [4]:
from oceanbench._src.datasets.base import XRDABatcher

In [5]:

x = CoordinateAxis.init_from_limits(x_min=1, x_max=360, dx = 1)
rng = np.random.RandomState(seed=123)
data = rng.randn(*x.data.shape)

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"
    
variable = Variable1D(data=data, x=x.data, name="ssh")
da = asdataarray(variable)
da.x

<xarray.DataArray 'x' (x: 360)>
array([1, 2, 3, ..., 358, 359, 360], dtype=object)
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In [6]:
patches = {"x": 30}
strides = {"x": 5}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: {'x': 360}
Patches:        {'x': 30}
Strides:        {'x': 5}
Num Batches:    {'x': 67}
Torch Dataset(size): 67


In [7]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 

all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)

In [8]:
import itertools

items = list(itertools.chain(*[all_batches]))
items_latent = list(itertools.chain(*[all_batches_latent]))

In [9]:
xrda_batches.coord_names

['x']

In [10]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones([30,])
dims_label = ["x"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w)
rec_da = reconstruct_from_items(items, dims_label, xrda_batches, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

100%|██████████████████████████████████████████| 67/67 [00:00<00:00, 457.39it/s]


## Test Case II - 2D Field

In [11]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [12]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50-0.05, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50-0.05, dlat=0.05)

da = SSH2D.init_from_axis(lon=lon_axis, lat=lat_axis)
da = asdataarray(da).to_dataset()
da

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 200)
Coordinates:
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Data variables:
    ssh      (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [13]:
patches = {"lat": 40, "lon": 20, }
strides = {"lat": 40, "lon": 20, }
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da.ssh,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: {'lat': 200, 'lon': 200}
Patches:        {'lat': 40, 'lon': 20}
Strides:        {'lat': 40, 'lon': 20}
Num Batches:    {'lat': 5, 'lon': 10}
Torch Dataset(size): 50


In [14]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(40, 20)

In [15]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,20))
dims_label = ["lat", "lon"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w)
rec_da = reconstruct_from_items(items, dims_label, xrda_batches, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 324.45it/s]


In [16]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,20))
dims_label_latent = ["lat", "lon", "z"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da.data)

100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 311.63it/s]


## Test Case III - 2D+T Field

In [17]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [18]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50-0.05, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50-0.05, dlat=0.05)
time_axis = TimeAxis.init_from_limits("2012-01-01", "2012-01-30", "1D")

da = SSH2DT.init_from_axis(lon=lon_axis, lat=lat_axis, time=time_axis)
da = asdataarray(da).to_dataset()
da

<xarray.Dataset>
Dimensions:  (time: 30, lat: 200, lon: 200)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-01-30
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Data variables:
    ssh      (time, lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [28]:
# patches = {"time": 10, "lat": 40, "lon": 20, }
patches = {"lon": 20, "time": 10, "lat": 40, }
strides = {"time": 10, "lat": 40, "lon": 20, }
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da.ssh,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('time', 30), ('lat', 200), ('lon', 200)])
Patches:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Strides:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Num Batches:    {'time': 3, 'lat': 5, 'lon': 10}
Torch Dataset(size): 150


In [34]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(10, 40, 20)

In [36]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((10,40,20))
dims_label = ["time", "lat", "lon"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w)
rec_da = reconstruct_from_items(items, dims_label, xrda_batches, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

100%|████████████████████████████████████████| 150/150 [00:00<00:00, 224.38it/s]


In [41]:
from oceanbench._src.datasets.utils import reconstruct_from_items

w = np.ones((10,40,20))
dims_label_latent = ["time", "lat", "lon", "z", "d"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N M", N=5, M=10), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

100%|█████████████████████████████████████████| 150/150 [00:03<00:00, 45.42it/s]


In [42]:
np.testing.assert_array_almost_equal(rec_da.isel(z=0, d=0).data, xrda_batches.da.data)